<a href="https://colab.research.google.com/github/Sanyam-oss/Online-Near-Duplicate-Detection/blob/main/Random_Forest_Classifier_Sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classifier Model to detect duplicate pairs**

Dataset : https://www.kaggle.com/competitions/quora-question-pairs/data

*Key features* about the data :

1. Data is labelled - `Binary Classification ()` problem

2. Labels have been supplied by humans, so truth labels are inherently subjective. Therefore, the `labels` on this dataset should be taken as informed and `not 100% accurate`.

---

**Problem Statement** 

`Given two questions we have to classify whether they are duplicates or not`.

---

**Background**

Currently, Quora uses a `Random Forest model` to identify duplicate questions.

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from collections import OrderedDict
import operator
import nltk
from nltk import bigrams, trigrams, word_tokenize
from collections import Counter, defaultdict

import re
from bs4 import BeautifulSoup

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
def _preprocess(q):     # Based on some observations from the EDA and Kaggle articles
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'billion ')
    q = q.replace(',000,000 ', 'million ')
    q = q.replace(',000 ', 'thousand ')

    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q
    

In [21]:
print(_preprocess("I've already seen that movie?"))

i have already seen that movie


In [22]:
new_df = pd.read_csv("drive/MyDrive/Quora_Dataset/qqd_features.csv", index_col=False)

new_df = new_df.loc[:, ~new_df.columns.str.contains('^Unnamed')]

In [23]:
new_df.isnull().sum()

id                       0
qid1                     0
qid2                     0
question1               16
question2               11
is_duplicate             0
len1                     0
len2                     0
word1                    0
word2                    0
common_words             0
word_total               0
word_share               0
cwc_min                  0
cwc_max                  0
csc_min                  0
csc_max                  0
ctc_min                  0
ctc_max                  0
last_word_eq             0
first_word_eq            0
abs_len_diff             0
mean_len                 0
longest_substr_ratio     0
fuzz_ratio               0
fuzz_partial_ratio       0
token_sort_ratio         0
token_set_ratio          0
dtype: int64

In [24]:
new_df = new_df.dropna()
print(new_df.shape)

(404264, 28)


In [25]:
ques_df = new_df[['question1','question2']]

final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])

print(final_df.shape)
final_df.head()

(404264, 23)


,is_duplicate,len1,len2,word1,word2,common_words,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,0,65,56,14,12,11,12,0.92,0.999980,0.833319,...,0.785709,0.0,1.0,2.0,13.0,0.982456,93,100,93,100
1,0,50,87,10,15,8,18,0.44,0.799984,0.399996,...,0.466664,0.0,1.0,5.0,12.5,0.588235,66,74,63,86
2,0,72,58,14,10,4,20,0.20,0.399992,0.333328,...,0.285712,0.0,1.0,4.0,12.0,0.169492,43,46,63,63
3,0,49,58,11,12,1,21,0.05,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,11.5,0.040000,9,11,25,28
4,0,75,38,13,7,4,17,0.24,0.399992,0.199998,...,0.307690,0.0,1.0,6.0,10.0,0.153846,35,55,47,67


In [26]:
ques1 = list(ques_df['question1']) 
ques2 = list(ques_df['question2'])


In [27]:
count = 0
sentences = []

for i in ques1:
  if  type(i) != type("") or len(i) <= 0 :
    print(i)
    count+=1
  else:
    sentences.append(i)

for i in ques2:
  if  type(i) != type("") or len(i) <= 0 :
    count+=1
  else:
    sentences.append(i)

print(count, " sentences removed.")

0  sentences removed.


In [28]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [30]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [31]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
embeddings_1 = np.array(model(ques1[:100000]))
embeddings_2 = np.array(model(ques1[100000:200000]))
embeddings_3 = np.array(model(ques1[200000:300000]))
embeddings_4 = np.array(model(ques1[300000:]))

In [33]:
embeddings_ques1 = np.concatenate((embeddings_1, embeddings_2, embeddings_3, embeddings_4), axis=0)
embeddings_ques1.shape

(404264, 512)

In [34]:
embeddings_1 = np.array(model(ques2[:100000]))
embeddings_2 = np.array(model(ques2[100000:200000]))
embeddings_3 = np.array(model(ques2[200000:300000]))
embeddings_4 = np.array(model(ques2[300000:]))

embeddings_ques2 = np.concatenate((embeddings_1, embeddings_2, embeddings_3, embeddings_4), axis=0)
embeddings_ques2.shape

(404264, 512)

In [35]:
del embeddings_1
del embeddings_2
del embeddings_3
del embeddings_4

In [36]:
temp_df1 = pd.DataFrame(embeddings_ques1, index= ques_df.index)
temp_df2 = pd.DataFrame(embeddings_ques2, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape 

(404264, 1024)

In [37]:
del temp_df1
del temp_df2
del embeddings_ques1
del embeddings_ques2

In [38]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(404264, 1047)


,is_duplicate,len1,len2,word1,word2,common_words,word_total,word_share,cwc_min,cwc_max,...,502,503,504,505,506,507,508,509,510,511
0,0,65,56,14,12,11,12,0.92,0.999980,0.833319,...,0.025082,0.081119,0.032776,0.050153,0.015376,0.072442,-0.049830,-0.017214,-0.006023,-0.012453
1,0,50,87,10,15,8,18,0.44,0.799984,0.399996,...,-0.028247,0.070963,0.001444,0.030775,-0.008241,-0.019384,-0.030941,-0.032340,-0.033636,-0.085151
2,0,72,58,14,10,4,20,0.20,0.399992,0.333328,...,-0.061788,0.080295,0.021691,-0.042301,-0.016445,-0.064183,-0.045031,-0.034261,-0.014464,-0.020912
3,0,49,58,11,12,1,21,0.05,0.000000,0.000000,...,0.076951,0.022152,0.029622,-0.024408,-0.031461,-0.068382,-0.022206,0.006905,0.000883,0.046865
4,0,75,38,13,7,4,17,0.24,0.399992,0.199998,...,-0.051983,0.082345,0.008434,0.030898,0.055961,-0.054330,-0.001506,-0.020883,0.027199,0.020707


In [39]:
sz = 250000

In [40]:
final_df = final_df[:sz]

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8228

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[5208, 1068],
       [1162, 2562]])

In [44]:
import pickle

pickle.dump(rf,open('rf_model.pkl','wb'))

In [50]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

!pip install Distance
import distance

!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def test_common_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return len(w1 & w2)

def test_total_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" "))) 
    return (len(w1|w2))

def test_fetch_token_features(q1,q2):
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

def test_fetch_length_features(q1,q2):
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

def test_fetch_fuzzy_features(q1,q2):
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 180 kB 7.6 MB/s 
  Created wheel for Distance: filename=Distance-0.1.3-py3-none-any.whl size=16276 sha256=e8f169dba62c5f865da1703cad7812c4be573e3d2d8e8c2349937de0558db92f
  Stored in directory: /root/.cache/pip/wheels/b2/10/1b/96fca621a1be378e2fe104cfb0d160bb6cdf3d04a3d35266cc
Successfully built Distance
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [79]:
def query_generator(inpq1,inpq2):
    
    input_query = []
    
    # preprocess
    q1 = _preprocess(inpq1)
    q2 = _preprocess(inpq2)
    
    input_query.append(len(q1))
    input_query.append(len(q2))
    
    input_query.append(len(q1.split(" ")))
    input_query.append(len(q2.split(" ")))
    
    input_query.append(test_common_words(q1,q2))
    input_query.append(test_total_words(q1,q2))
    input_query.append(round(test_common_words(q1,q2)/test_total_words(q1,q2),2))
    
    # fetch token features
    token_features = test_fetch_token_features(q1,q2)
    input_query.extend(token_features)
    
    # fetch length based features
    length_features = test_fetch_length_features(q1,q2)
    input_query.extend(length_features)
    
    # fetch fuzzy features
    fuzzy_features = test_fetch_fuzzy_features(q1,q2)
    input_query.extend(fuzzy_features)
    

    q1_embeddings = model([q1])
    q2_embeddings = model([q2])
     
    return np.hstack((np.array(input_query).reshape(1,22), q1_embeddings, q1_embeddings))

In [117]:
q1 = 'How can I be a good geologist?'
q2 = 'What should I do to be a great geologist?'
q3 = 'Which is the strongest bone in human body'
q4 = 'How many bones are there in human body'

In [118]:
features_query = query_generator(q3,q4) # 512 + 512 + 22 features
features_query.shape

1


(1, 1046)

In [119]:
rf.predict(features_query)

array([0])